## AGGREGATION DATA AND MAKE TABLES FOR BOTH DOMAINS DOCTORS AND NURSING


In [1]:
%%sql
CREATE OR REPLACE TABLE gold_doctors_data
USING DELTA
AS
SELECT e.patient_id,
e.length_of_stay,
f.hour_from_admission,
f.wbc_count,
f.lactate,
f.creatinine,
f.crp_level,
f.hemoglobin,
v.heart_rate,
v.respiratory_rate,
v.spo2_pct,
v.temperature_c,
v.systolic_bp,
v.diastolic_bp,
v.max_heart_rate,
v.max_respiratory_rate,
max_spo2_pct,
v.max_temperature_c,
v.max_systolic_bp,
v.max_diastolic_bp,
CASE 
  WHEN f.lactate > 3 AND v.spo2_pct < 92 THEN 1
  ELSE 0
END AS critical_hour_flag
FROM silver_patients e
LEFT JOIN silver_labs f
ON e.patient_id=f.patient_id
LEFT JOIN silver_vitals v
ON e.patient_id=v.patient_id AND f.hour_from_admission = v.hour_from_admission
ORDER by patient_id asc , hour_from_admission ASC


StatementMeta(, 0bfa2c3f-0758-43da-ba42-6fe3828e6a0e, 2, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [2]:
%%sql
CREATE OR REPLACE TABLE gold_nursing_data
USING DELTA
AS
SELECT e.patient_id,
e.length_of_stay,
v.hour_from_admission,
v.heart_rate,
v.respiratory_rate,
v.spo2_pct,
v.temperature_c,
v.systolic_bp,
v.diastolic_bp,
v.nurse_alert,
CASE
      WHEN v.max_heart_rate > 130 OR v.max_spo2_pct < 92 THEN 1
      ELSE 0
    END AS immediate_attention_flag,

    CASE
      WHEN v.max_temperature_c > 38 THEN 1
      ELSE 0
    END AS fever_flag
FROM silver_patients e 
LEFT JOIN silver_vitals v
ON e.patient_id = v.patient_id
ORDER BY patient_id ASC , hour_from_admission ASC

StatementMeta(, 0bfa2c3f-0758-43da-ba42-6fe3828e6a0e, 3, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>